In [1]:
%load_ext autoreload
%autoreload 2

# Simplificando um acórdão

In [2]:
from langchain_ollama import ChatOllama
from llms import get_llama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_core.documents.base import Document
from langchain_core.prompts import PromptTemplate

from config import Config
Config.OLLAMA_BASE_URL

'http://10.10.0.99:11434'

In [3]:
def split_documents(file_path, page_start=None, page_end=None):
    if file_path.endswith('.pdf'):
        loader = PyMuPDFLoader(file_path=file_path)
        doc = loader.load()[page_start:page_end]
    else:
        loader = TextLoader(file_path)
        docs = loader.load()
        doc = [Document(page) for page in docs[0].page_content.split('\n-----\n')[page_start:page_end]]
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=Config.SPLITTER_CHUNK_SIZE,
        chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [24]:
from pprint import pprint
from langchain_ollama import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS


def stuff(docs, host, results, key, prompt):
    llm: ChatOllama = get_llama(host=host, model=0, log_callbacks=True)

    embeddings_model = OllamaEmbeddings(
        base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
        model=Config.OLLAMA_EMBEDDINGS_MODEL
    )
    vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings_model)
    retriever = vectorstore.as_retriever()

    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    chunks = []

    for chunk in retrieval_chain.stream({
        'input': 'Comece.'
    }):
        answer = chunk.get('answer', '')
        chunks.append(answer)
        print(answer, end='', flush=True)
    print()
    result = "".join(chunks)

    results[key]['response'] = result
    results[key]['prompt'] = [
        str(prompt),
    ]

    del(vectorstore)
    del(retriever)
    return result

In [59]:
from semantic_similarity import get_similarity_score


def evaluate(summary, original):
    original_content = "\n".join([page.page_content for page in original])
    return {
        'cosine': get_similarity_score(summary, original_content, method='cosine'),
        'bertscore': get_similarity_score(summary, original_content, method='bertscore'),
    }

In [5]:
results = {
    "cabecalho": {},
    "relatorio": {},
    "voto": {},
    "decisao": {},
}

In [6]:
simplificados = {}

In [7]:
docs = [
    "documentos/acordaos/0600012-49_REl_28052024_1.txt",
    "documentos/acordaos/0600108-33_RE_16122021_1.pdf",
    "documentos/acordaos/0600376-38_REl_10042023_1.pdf",
    "documentos/acordaos/0600059-75_REl_03102023_1.pdf",
    "documentos/acordaos/0602653-80_DR_29092022_1.pdf",
    "documentos/acordaos/0602658-05_REl-DR_30092022_1.pdf",
    "documentos/acordaos/0600067-56_REl_27102022_1.pdf",
    "documentos/acordaos/0602725-67_DR_30092022_1.pdf",
    "documentos/acordaos/0600085-38_RE_25042022_1.pdf",
    "documentos/acordaos/0602730-89_DR_29092022_1.pdf",
]
file_path = docs[0]

## Extraindo o cabeçalho

In [8]:
start_page = 0
end_page = 1
doc = split_documents(file_path, start_page, end_page)

### Prompt do cabeçalho

In [9]:
cabecalho_prompt = PromptTemplate.from_template(
    """
    Extraia do texto encontrado no contexto somente as seguintes informações:

    -   Número do processo (ou número do recurso eleitoral)
    -   Origem
    -   Relator
    -   Recorrente
    -   Advogados(as)
    -   Recorrido
    -   Assunto

    Formate sua resposta como uma lista, assim:

    -   Número do processo: Número do processo
    -   Origem: Origem
    -   Relator: Relator
    -   Recorrente: Recorrente
    -   Advogados: Advogados(as)
    -   Recorrido: Recorrido
    -   Assunto: Assunto, em 2 ou 3 palavras

    ### Exemplo de entrada:

    ```
    PODER JUDICIÁRIO
    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ

    RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033
    ORIGEM: CANINDÉ/CE
    RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,
    LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    RECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL
    ```

    ### Exemplo de saída:

    -   Número do processo: 0600012-49.2024.6.06.0033
    -   ORIGEM: CANINDÉ/CE
    -   RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    -   RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    -   ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A, LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    -   RECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL

    ### Contexto

    ```
    {context}
    ```

    ### Fim do contexto

    ### Resposta
    """
)

### Recebendo a resposta

In [10]:
result = stuff(doc, 0, results, 'cabecalho', cabecalho_prompt)

Token indices sequence length is longer than the specified maximum sequence length for this model (1466 > 1024). Running this sequence through the model will result in indexing errors



Prompt: Human: 
    Extraia do texto encontrado no contexto somente as seguintes informações:

    -   Número do processo (ou número do recurso eleitoral)
    -   Origem
    -   Relator
    -   Recorrente
    -   Advogados(as)
    -   Recorrido
    -   Assunto

    Formate sua resposta como uma lista, assim:

    -   Número do processo: Número do processo
    -   Origem: Origem
    -   Relator: Relator
    -   Recorrente: Recorrente
    -   Advogados: Advogados(as)
    -   Recorrido: Recorrido
    -   Assunto: Assunto, em 2 ou 3 palavras

    ### Exemplo de entrada:

    ```
    PODER JUDICIÁRIO
    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ

    RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033
    ORIGEM: CANINDÉ/CE
    RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,
    LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    RECORRIDO: MINISTÉRIO PÚBLICO ELEITORA

## Resumindo o relatório

In [11]:
start_page = 2
end_page = 5
doc = split_documents(file_path, start_page, end_page)

### Prompt do relatório

In [12]:
relatorio_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:
    Responda com o texto simplificado do acórdão, e nada mais.

    ### FORMATO
    ```
    **Relatório (O Caso)**: Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    Informações iniciais do processo analisado pelo juiz: indique o que o autor do recurso pediu e o que o réu alegou para se defender.
    Decisão do juiz no processo inicial: Apresente a decisão do juiz no processo inicial, descrevendo o que o juiz decidiu e as justificativas legais usadas.
    Artigos de lei e fundamentos jurídicos relevantes: cite artigos de lei e fundamentos jurídicos relevantes..
    Quem recorreu e o que alegou: indique quem recorreu à decisão e o que alegou para recorrer.

    Caixa de texto explicativa com termos jurídicos relevantes para a compreensão do assunto principal: forneça definições e explicações simples de termos, expressões ou assuntos jurídicos relevantes para a compreensão do assunto principal. Exemplo: “Propaganda antecipada negativa: A propaganda eleitoral antecipada negativa acontece quando, antes de 16 de agosto do ano eleitoral (art. 36 da Lei nº 9.504/1997), alguém faz críticas para prejudicar adversários políticos e influenciar eleitores. Essa prática é proibida e pode resultar em multa”.
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO
    """
)

### Recebendo a resposta

In [13]:
result = stuff(doc, 0, results, 'relatorio', relatorio_prompt)


Prompt: Human: 
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:
    Responda com o texto simplificado do acórdão, e nada mais.

    ### FORMATO
    ```
    **Relatório (O Caso)**: Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    Informações iniciais do processo analisado pelo juiz: indique o que o autor do recurso pediu e o que o réu alegou para se defender.
    D

# Avaliando resposta

In [60]:
print(evaluate(results['relatorio']['response'], doc))

{'cosine': 0.7023019160910261, 'bertscore': 0.5260257720947266}


## Refinando a resposta

In [61]:
import importlib
import postprocessing

importlib.reload(postprocessing)

<module 'postprocessing' from '/home/rafael-albuquerque/LIODS/repos/simplificacao/postprocessing.py'>

In [62]:
post = postprocessing.postprocess(results['relatorio']['response'], file_path, start_page, end_page)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [63]:
pprint(post)

('**Resumo Refinado**\n'
 '\n'
 'O Tribunal Regional Eleitoral do Ceará (TRE-CE) negou provimento ao recurso '
 'eleitoral apresentado por Antônio Ilomar Vasconcelos Cruz, candidato a um '
 'cargo eletivo, que alegava que sua pré-candidatura foi prejudicada pela '
 'propaganda eleitoral antecipada realizada por Francisco Jardel Rodrigues de '
 'Souza.\n'
 '\n'
 'O juiz havia decidido que o ato do candidato Ilomar foi caracterizado como '
 'propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, '
 'da Lei 9.504/97. O réu argumentou que não havia pedido explícito de votos ou '
 'conduta que prejudicasse a igualdade entre os candidatos.\n'
 '\n'
 'O TRE-CE decidiu que o ato do candidato Ilomar foi caracterizado como '
 'propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, '
 'da Lei 9.504/97. O Tribunal considerou que o ato do candidato Ilomar foi '
 'prejudicial à igualdade entre os candidatos e aplicou a penalidade prevista '
 'na lei.\n'
 '\n'


In [64]:
print(evaluate(post, doc))

{'cosine': 0.853852466625631, 'bertscore': 0.5514101982116699}


## Resumindo o voto

In [17]:
start_page = 5
end_page = 11
doc = split_documents(file_path, start_page, end_page)

### Prompt do voto

In [18]:
voto_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:
    Responda com o texto simplificado do acórdão, e nada mais.

    ### FORMATO
    ```
    **Voto (Argumentação/ Motivação/ Fundamentação)**: Apresente as razões jurídicas que embasaram a decisão do Tribunal, destacando, em 2 parágrafos, com no máximo 3 linhas, de texto contínuo.

    Princípios legais aplicados: quais foram os princípios legais aplicados.
    Interpretação do Tribunal: A interpretação dada à legislação.
    Argumentos relevantes: Argumentos relevantes usados na motivação.**

    (Use uma linguagem simples e evite jargões técnicos sempre que possível)

    Fundamentação de Fato e de Direito: Explique os fatos relevantes para a decisão e apresente a fundamentação jurídica, citando artigos de lei, precedentes jurisprudenciais que embasam a decisão do Tribunal.
    Conclusão e Decisão: Conclusão do Voto.
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO
    """
)

### Recebendo a resposta

In [19]:
result = stuff(doc, 0, results, 'voto', voto_prompt)


Prompt: Human: 
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:
    Responda com o texto simplificado do acórdão, e nada mais.

    ### FORMATO
    ```
    **Voto (Argumentação/ Motivação/ Fundamentação)**: Apresente as razões jurídicas que embasaram a decisão do Tribunal, destacando, em 2 parágrafos, com no máximo 3 linhas, de texto contínuo.

    Princípios legais aplicados: quais foram os princípios legais aplicados.
    Interpretação do Tribunal: A interpretação dada à legislação.
    Argumentos relevantes

## Resumindo a decisão

In [47]:
import split_documents as splitdocuments

start_page = 11
end_page = 12
doc = splitdocuments.split_documents(file_path, start_page, end_page, chunk_size=200, chunk_overlap=0)

### Prompt da decisão

In [48]:
decisao_prompt = PromptTemplate.from_template(
    """Classifique o resultado do recurso como "RECURSO ACEITO", "RECURSO NEGADO" ou "RECURSO PARCIALMENTE ACEITO". Responda com a classificação e justifique sua resposta.

Texto:
{context}

Classificação:"""
)

### Recebendo a resposta

In [50]:
decisao_verbosa = stuff(doc, 0, results, 'decisao', decisao_prompt)


Prompt: Human: Classifique o resultado do recurso como "RECURSO ACEITO", "RECURSO NEGADO" ou "RECURSO PARCIALMENTE ACEITO". Responda com a classificação e justifique sua resposta.

Texto:
COMPOSIÇÃO: DESEMBARGADOR  ELEITORAL RAIMUNDO NONATO  SILVA SANTOS
(PRESIDENTE),  DESEMBARGADOR  ELEITORAL  FRANCISCO  GLADYSON  PONTES,

DANIEL CARVALHO CARNEIRO, DESEMBARGADOR ELEITORAL LUCIANO NUNES MAIA
FREIRE, DESEMBARGADOR ELEITORAL SUBSTITUTO ROGÉRIO FEITOSA CARVALHO MOTA.

DECISÃO: Acordam os Membros do Tribunal Regional Eleitoral do Ceará, por unanimidade, em conhecer
do recurso para negar-lhe provimento, nos termos do voto do Relator.

DESEMBARGADOR ELEITORAL GLÊDISON MARQUES FERNANDES, DESEMBARGADOR
ELEITORAL FRANCISCO ÉRICO CARVALHO SILVEIRA, DESEMBARGADOR ELEITORAL

Classificação:

Tokens: 337
A classificação do resultado do recurso é "RECURSO NEGADO".

Justificativa:

* O texto indica que a decisão do Tribunal Regional Eleitoral do Ceará é de negar provimento ao recurso, o que é express

In [54]:
decisao_pontual_prompt = PromptTemplate.from_template(
    """Você receberá um texto no seguinte formato:

    ```
    Classificação: {{Classificação}}

    Justificativa:

    {{Justificativa}}
    ```

    Texto:

    {texto}

    Responda apenas com o conteúdo de "{{Classificação}}".

    Classificação:
    """
)

In [55]:
llm = get_llama(log_callbacks=True)
decisao_chain = decisao_pontual_prompt | llm

results['decisao']['response'] = decisao_chain.invoke({'texto': decisao_verbosa}).content
print(results['decisao']['response'])


Prompt: Human: Você receberá um texto no seguinte formato:

    ```
    Classificação: {Classificação}

    Justificativa:

    {Justificativa}
    ```

    Texto:

    A classificação do resultado do recurso é "RECURSO NEGADO".

Justificativa:

* O texto indica que a decisão do Tribunal Regional Eleitoral do Ceará é de negar provimento ao recurso, o que é expresso na frase "Acordam os Membros do Tribunal Regional Eleitoral do Ceará, por unanimidade, em conhecer do recurso para negar-lhe provimento".
* Além disso, a decisão é tomada por unanimidade, o que significa que todos os membros do tribunal concordaram na negação do recurso.
* Não há menção a aceitação parcial ou a necessidade de revisão da decisão, o que sugere que o recurso foi completamente negado.

    Responda apenas com o conteúdo de "{Classificação}".

    Classificação:
    

Tokens: 327
RECURSO NEGADO.


# Visualizando Resumo

In [56]:
acordao = f"""# Tribunal Regional Eleitoral do Ceará

{results['cabecalho']['response']}

## Relatório

{results['relatorio']['response']}

## Voto

{results['voto']['response']}

## Decisão

{results['decisao']['response']}
"""
print(acordao)

# Tribunal Regional Eleitoral do Ceará

-   Número do processo: 0600012-49.2024.6.06.0033
-   Origem: CANINDÉ/CE
-   Relator: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
-   Recorrente: ANTÔNIO ILOMAR VASCONCELOS CRUZ
-   Advogados(as): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A, LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
-   Recorrido: MINISTÉRIO PÚBLICO ELEITORAL

## Relatório

**Relatório (O Caso)**: O caso envolve um recurso eleitoral apresentado pelo candidato Ilomar, que alega que sua pré-candidatura foi prejudicada pela realização de propaganda eleitoral antecipada. O juiz decidiu que o ato do candidato Ilomar foi caracterizado como propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, da Lei 9.504/97.

**Informações iniciais do processo**: O autor do recurso pediu a anulação da decisão do juiz que considerou a propaganda eleitoral antecipada e a aplicação da penalidade. O réu alegou que não havia pedido explícito de votos ou conduta q

In [57]:
from semantic_similarity import get_similarity_score


original_content = open(file_path, 'r').read()
evaluation = {
    'cosine': get_similarity_score(acordao, original_content, 'cosine'),
    'bertscore': get_similarity_score(acordao, original_content, 'bertscore'),
}

pprint(evaluation)

{'bertscore': 0.749885618686676, 'cosine': 0.36415524971935453}


In [21]:
simplificar_relatorio_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de documentos legais. Sua missão é reescrever documentos legais em linguagem simplificada, visando o entendimento de pessoas não familiarizadas com o linguajar jurídico. Mantenha a estrutura dos documentos que receber, preservando os títulos e subtítulos, exceto a conclusão, entre outros componentes estruturais do texto. Preserve as informações principais do texto original de forma fiel.

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO

    Texto simplificado:
    """
)

In [22]:
simplificar_chain = simplificar_relatorio_prompt | get_llama(0)
result = simplificar_chain.invoke({'context': results['relatorio']['response']})

KeyboardInterrupt: 

In [16]:
simplificados['relatorio'] = result.content
print(simplificados['relatorio'])

**Relatório (O Caso)**: Um pré-candidato a prefeito em Canindé realizou uma pré-campanha com estrutura robusta, incluindo palco, iluminação e telão em LED. O evento foi divulgado nas redes sociais e teve pedido explícito de votos.

**Informações iniciais do processo**: O Ministério Público Eleitoral alegou que essas condutas constituem propaganda eleitoral antecipada, proibida pela lei eleitoral. O juiz da 33ª Zona - Canindé/CE concordou e condenou o pré-candidato ao pagamento de multa no valor de R$ 15.000.

**Decisão do juiz**: O juiz julgou parcialmente procedente a representação eleitoral por propaganda antecipada e condenou o pré-candidato ao pagamento de multa, com fulcro no art.36, § 3º, da Lei 9.504/97.

**Artigos de lei relevantes**: O juiz aplicou o art. 36, § 3º, da Lei 9.504/97, que proíbe a propaganda eleitoral antecipada.

**Recorrido e fundamentos jurídicos**: O pré-candidato recorreu à decisão do juiz e alegou que as condutas realizadas durante a pré-campanha estavam de

In [ ]:
simplificar_prompt = PromptTemplate.from_template("""
    Simplifique o texto, mantendo sua estrutura original e seguindo as seguintes diretivas:

    1. Utilize sempre uma linguagem simples e direta, evitando termos técnicos desnecessários. Caso seja imprescindível usar um termo jurídico específico, certifique-se de explicá-lo claramente.
    2. Mantenha o número de parágrafos e linhas conforme indicado para cada seção.
    3. Assegure-se de que as informações jurídicas sejam precisas e que a fundamentação legal esteja diretamente relacionada ao caso em discussão.
    4. Siga as regras de linguagem simples: seja empático e cordial, use frases curtas, escreva na ordem direta e em voz ativa, coloque as informações mais importantes no início e utilize verbos de ação.
    5. Revise o texto para garantir que seja claro e acessível, evitando siglas, jargões e termos técnicos. Confirme se todas as diretrizes e restrições foram cumpridas.
    6. Adote uma escrita humanizada, procurando aproximar o texto da forma como um ser humano se expressaria, para tornar a comunicação mais próxima e compreensível.

    ### CONTEXTO

    ```
    {context}
    ```

    ### RESPOSTA
""")

In [ ]:
simplificar_chain = simplificar_prompt | get_llama(0)

In [ ]:
result = simplificar_chain.invoke({'context': acordao})

In [ ]:
pprint(result.content)